In [162]:
import os
import pandas as pd
import numpy as np

dateCol=["Release Date","Customer Target XFD (CTXFD)","NB Requested XFD (NBRXFD)","Orig Fact Cfm XFD (OCXFD)","NB Curr Req XFD","Vendor Current Req XFD",
         "Curr Fact Cfm XFD (CCXFD)","SI Date","Start Production Dt","Customer Request Date","Original Customer Target XFD (OCTXFD)","Current Customer Target XFD (CCTXFD)",
         "Exp XFD","Booking Request Send Date","Production Complete Date","Intro Date (PPM)","Receipt Date"]

csvs=[x for x in os.listdir(".") if x.lower().endswith("csv")]

vsCol=["NB Requested XFD (NBRXFD)","Orig Fact Cfm XFD (OCXFD)","Customer Target XFD (CTXFD)","SI Date"]

rCol=vsCol[:]
rCol.extend([x for x in cj.columns if "vs" in x.lower() and "/" not in x])
rCol

def inspect(dfp,less=False):
    dfcol=dfp.columns
    susCol=[x for x in dfcol if "vs" in x.lower() and "/" not in x]
    filtered=dfp[dfp[susCol].nunique(axis=1)>1]
    del dfp
    if less==True:
        # return dfp[(dfp[susCol].abs().gt(250).any(axis=1)) & (dfp[susCol].nunique(axis=1)>1)][susCol]
        return filtered[filtered[susCol].abs().gt(250).any(axis=1)][susCol]
    else:
        return filtered[filtered[susCol].abs().gt(250).any(axis=1)]

# 0709 end todo:
1. create index
2. confirm style &rarr; jan-dec, all subordinary item give seq 1-12
3. check wrong dates in (SI vs NBRXFD) or logic
4. re-create vars
5. monthly

multi level reg.

In [19]:
# for i in [x for x in os.listdir() if x.endswith("csv")]:
#     globals()[i.split(".")[0]]=pd.read_csv(i,thousands=",")

# cj["styleSeq"]=cj.groupby("orderID").cumcount()+1
# cj[["orderID","styleSeq"]]


# check primary key:
# cj.nunique().to_dict()

# `CJ1`:

In [28]:
#orderID, product, orderqua:
cj=pd.read_csv("CJ1.csv",thousands=",").rename({"Order#":"orderID","GBU (PPM)":"product","Quantity":"orderQnty"}, axis=1)
cj.loc[cj[cj["SI Month"]=="2022/00"].index,"SI Month"]="2021/12"

cj[dateCol]=cj[dateCol].apply(pd.to_datetime)
cj.head(1)

,SI Month,Fty Country (COO),Factory Abbr,INTL/NA,Mkt Region,Market,product,Line Plan Business (PPM),orderID,PO Ln#,...,Made / Purchased,SMU?,Wid/Col,Feature Technology,World Trade Commodity,World Trade Country Code,Local Product Group Minor,Local Product Minor,Product Display Name,Closure Type
0,2021/02,CHINA,CJ,INTL,APAC,TAIWAN,Core Athletic,Core Athletic,7092294,7092294-1,...,P,NaN,D,"GORE-TEX , Synthetic/Mesh Upper, TRUFUSE, WALK...",NaN,NaN,152,Womens Walking,880 Gore-Tex,Lace


In [30]:
# @var: styleseq
cj["SI Month"] = pd.to_datetime(cj["SI Month"], format='%Y/%m')
cj=cj.sort_values(by=["Style","SI Month"])
cj['styleseq'] = cj.groupby('Style')["SI Month"].rank(method='dense').astype(int)
cj["SI Month"] = cj["SI Month"].dt.strftime('%Y/%m')
cj.loc[cj.Style==np.random.choice(cj.Style), ["SI Month","styleseq"]]

,SI Month,styleseq
89,2021/01,1
163,2021/02,2
200,2021/02,2


In [31]:
cj.loc[cj.Style==np.random.choice(cj.Style), ["SI Month","styleseq"]]

,SI Month,styleseq
42,2021/01,1
81,2021/03,2
288,2021/03,2
435,2021/05,3
726,2021/05,3
777,2021/06,4
1044,2021/08,5
1045,2021/08,5
1091,2021/11,6
1099,2021/11,6


In [63]:
# cj check:
# cj[cj["SI vs CTXFD"]>180][[x for  x in cj.columns if "CTXFD" in x][:3]]
cj.loc[2004,[x for  x in cj.columns if "CTXFD" in x][:3]]="4/3/2022"
cj.loc[325,"NB Requested XFD (NBRXFD)"]="2021-03-09"
cj.loc[392,"NB Requested XFD (NBRXFD)"]="2021-03-16"

In [67]:
# ?OnTime
for i in zip(["SI vs CTXFD Early/On-time/Late","SI vs NBRXFD Early/On-time/Late",  "SI vs OCXFD Early/On-time/Late"],["iOnTime","nOnTime","cOnTime"]):
    cj[i[1]]=cj[i[0]].map( {'Early':3, 'On-time':2, 'Late':1} )

In [ ]:
#ld_timei, ld_timen, ld_timec, prd_dayso, prd_timeo, introtime:

cj["ld_timei"]=(cj["Customer Target XFD (CTXFD)"]-cj["Release Date"]).dt.days #ideal
cj["ld_timen"]=(cj["NB Requested XFD (NBRXFD)"]-cj["Release Date"]).dt.days #brand
cj["ld_timec"]=(cj["Orig Fact Cfm XFD (OCXFD)"]-cj["Release Date"]).dt.days #confirm
cj["prd_dayso"]=(cj["Production Complete Date"]-cj["Start Production Dt"]).dt.days 
cj["prd_timeo"]=(cj["Start Production Dt"]-cj["Release Date"]).dt.days 
cj["introtime"]=(cj["Start Production Dt"]-cj["Intro Date (PPM)"]).dt.days 

# `FTV2`:

In [72]:
ftv=pd.read_csv("FTV2.csv")

C:\Users\JselZ\AppData\Local\Temp\ipykernel_13768\132380678.py:1: DtypeWarning: Columns (62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  ftv=pd.read_csv("FTV2.csv")


In [157]:
# only 1
ftv.loc[788,"NB Requested XFD (NBRXFD)"]="2021/1/25"

In [158]:
inspect(ftv,less=0)[rCol]

,Customer Target XFD (CTXFD),NB Requested XFD (NBRXFD),Orig Fact Cfm XFD (OCXFD),SI Date,SI vs NBRXFD,SI vs OCXFD,SI vs CTXFD
788,2020/11/24,2021/1/25,2021/1/25,2021/1/23,364,-2,60
5667,2021/7/12,2021/11/1,2021/11/1,2022/3/31,150,150,262
5839,2021/7/5,2021/11/1,2021/11/1,2022/3/31,150,150,269
5881,2021/6/14,2021/11/1,2021/11/1,2022/3/23,142,142,282
6215,2021/6/27,2021/11/1,2021/11/1,2022/3/9,128,128,255
6529,2021/6/7,2021/9/27,2021/9/27,2022/2/18,144,144,256
6540,2021/6/7,2021/10/4,2021/10/4,2022/2/18,137,137,256
6548,2021/7/9,2021/11/22,2021/11/22,2022/3/25,123,123,259
6549,2021/7/9,2021/11/22,2022/3/25,2022/3/25,123,0,259
6740,2021/7/1,2021/10/18,2021/10/18,2022/3/11,144,144,253


# `PWI2`

In [106]:
pwi=pd.read_csv("PWI2.csv",thousands=",")

C:\Users\JselZ\AppData\Local\Temp\ipykernel_13768\647758339.py:1: DtypeWarning: Columns (51,63) have mixed types. Specify dtype option on import or set low_memory=False.
  pwi=pd.read_csv("PWI2.csv",thousands=",")


In [171]:
pwi.loc[[7321, 7548, 7550, 7551, 7552, 7566, 7946], "NB Requested XFD (NBRXFD)"]="9/8/2021"
pwi.loc[22657, "Orig Fact Cfm XFD (OCXFD)"]="1/10/2023"

In [172]:
inspect(pwi)[rCol]

,NB Requested XFD (NBRXFD),Orig Fact Cfm XFD (OCXFD),Customer Target XFD (CTXFD),SI Date,SI vs NBRXFD,SI vs OCXFD,SI vs CTXFD
7321,9/8/2021,9/8/2021,9/30/2021,9/8/2021,1461,0,-22
7548,9/8/2021,9/8/2021,9/30/2021,9/8/2021,1461,0,-22
7550,9/8/2021,9/8/2021,9/28/2021,9/8/2021,1461,0,-20
7551,9/8/2021,9/8/2021,9/30/2021,9/8/2021,1461,0,-22
7552,9/8/2021,9/8/2021,9/30/2021,9/8/2021,1461,0,-22
7566,9/8/2021,9/8/2021,9/30/2021,10/1/2021,1484,23,1
7946,9/8/2021,11/15/2021,9/25/2021,9/12/2022,301,301,352
22657,1/10/2023,1/10/2023,1/11/2023,1/10/2023,0,-273,-1


# `PYV`

In [119]:
pyv=pd.read_csv("PYV.csv",thousands=",")

In [169]:
pyv.loc[378, "Customer Target XFD (CTXFD)"]="1/18/2021"

In [170]:
inspect(pyv)[rCol]

,NB Requested XFD (NBRXFD),Orig Fact Cfm XFD (OCXFD),Customer Target XFD (CTXFD),SI Date,SI vs NBRXFD,SI vs OCXFD,SI vs CTXFD
378,2/22/2021,2/22/2021,1/18/2021,2/22/2021,0.0,0,401
3292,8/23/2021,8/23/2021,6/21/2021,4/18/2022,238.0,238,301
3293,8/23/2021,8/23/2021,6/21/2021,4/12/2022,232.0,232,295
3294,8/23/2021,8/23/2021,6/21/2021,4/19/2022,239.0,239,302
3295,8/23/2021,8/23/2021,6/21/2021,4/16/2022,236.0,236,299
3311,8/16/2021,8/16/2021,6/7/2021,4/5/2022,232.0,232,302
3312,8/16/2021,8/16/2021,6/7/2021,4/21/2022,248.0,248,318
3395,8/16/2021,8/16/2021,6/14/2021,3/16/2022,212.0,212,275
3396,8/16/2021,8/16/2021,6/14/2021,3/16/2022,212.0,212,275
3397,8/16/2021,8/16/2021,6/14/2021,4/14/2022,241.0,241,304


# `SA1`

In [122]:
sa=pd.read_csv("SA1.csv",thousands=",")

C:\Users\JselZ\AppData\Local\Temp\ipykernel_13768\1505989697.py:1: DtypeWarning: Columns (62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  sa=pd.read_csv("SA1.csv",thousands=",")


In [167]:
sa.loc[11,"NB Requested XFD (NBRXFD)"]="1/11/2021"
sa.loc[13,"NB Requested XFD (NBRXFD)"]="1/4/2021"
sa.loc[[12526,12527],"NB Requested XFD (NBRXFD)"]="12/13/2022"

In [175]:
inspect(sa)[rCol]

,NB Requested XFD (NBRXFD),Orig Fact Cfm XFD (OCXFD),Customer Target XFD (CTXFD),SI Date,SI vs NBRXFD,SI vs OCXFD,SI vs CTXFD
4887,7/5/2021,7/5/2021,6/28/2021,3/26/2022,264,264,271
5194,7/19/2021,7/19/2021,6/7/2021,3/18/2022,242,242,284
5247,7/1/2021,7/1/2021,5/31/2021,2/7/2022,221,221,252
5447,6/7/2021,8/16/2021,7/1/2021,2/23/2022,261,191,237
5467,8/30/2021,8/30/2021,7/12/2021,4/29/2022,242,242,291
5536,8/30/2021,8/30/2021,6/28/2021,3/8/2022,190,190,253
5562,8/23/2021,8/23/2021,6/28/2021,3/7/2022,196,196,252
5587,7/19/2021,9/1/2021,6/27/2021,3/29/2022,253,209,275
6281,9/28/2021,9/28/2021,8/30/2021,5/19/2022,233,233,262
6480,10/11/2021,4/26/2022,10/4/2021,7/5/2022,267,70,274


# recalculate `vs` columns

In [174]:
for df in [cj,ftv,pwi,pyv,sa]:
    for group in zip(rCol[:3],rCol[-3:]):
        df[group[-1]]=(pd.to_datetime(df["SI Date"])-pd.to_datetime(df[group[0]])).dt.days 

# vars. creation (loop)

[all_orders.ipynb](./all_orders.ipynb)

In [176]:
cj

,SI Month,Fty Country (COO),Factory Abbr,INTL/NA,Mkt Region,Market,product,Line Plan Business (PPM),orderID,PO Ln#,...,World Trade Commodity,World Trade Country Code,Local Product Group Minor,Local Product Minor,Product Display Name,Closure Type,styleseq,iOnTime,nOnTime,cOnTime
1838,2022/07,CHINA,CJ,INTL,JAPAN,JAPAN,Lifestyle,Lifestyle,7344924,7344924-1,...,NaN,NaN,C81,Mens Sport Lifestyle,996,Lace,1,3,2,2
1839,2022/07,CHINA,CJ,INTL,JAPAN,JAPAN,Lifestyle,Lifestyle,7344925,7344925-1,...,NaN,NaN,C81,Mens Sport Lifestyle,996,Lace,1,3,2,2
1857,2022/07,CHINA,CJ,INTL,JAPAN,JAPAN,Lifestyle,Lifestyle,7348863,7348863-1,...,NaN,NaN,C81,Mens Sport Lifestyle,996,Lace,1,3,3,3
1709,2022/08,CHINA,CJ,INTL,APAC,THAILAND,Lifestyle,Lifestyle,7311119,7311119-1,...,NaN,NaN,C81,Mens Sport Lifestyle,996,Lace,2,2,2,2
1901,2022/08,CHINA,CJ,INTL,APAC,TAIWAN,Lifestyle,Lifestyle,7351430,7351430-1,...,NaN,NaN,C81,Mens Sport Lifestyle,996,Lace,2,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,2021/02,CHINA,CJ,INTL,CHINA,CHINA,Kids,Kids Lifestyle,7149443,7149443-1,...,NaN,NaN,334,Boys Sandals,NCLAY SANDAL,Sandals,1,3,3,3
428,2021/03,CHINA,CJ,INTL,CHINA,CHINA,Kids,Kids Lifestyle,7149439,7149439-1,...,NaN,NaN,334,Boys Sandals,NCLAY SANDAL,Sandals,2,1,3,3
1403,2021/12,CHINA,CJ,INTL,CHINA,CHINA,Kids,Kids Lifestyle,7285996,7285996-1,...,NaN,NaN,334,Boys Sandals,NCLAY SANDAL,Sandals,3,3,2,2
1406,2022/02,CHINA,CJ,INTL,CHINA,CHINA,Kids,Kids Lifestyle,7285999,7285999-1,...,NaN,NaN,334,Boys Sandals,NCLAY SANDAL,Sandals,4,2,2,2
